<img src="wam_banner_2023.png" width="700"/>

# Lab 1. Sounds and Harmonics

In this lab we will produce some sounds with their harmonics, in order to study their acoustic effects

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as ss
from math import pi, sin
from random import random
from matplotlib.ticker import LogFormatter
import simpleaudio as sa
import ipywidgets as widgets
fs = 44100        # 44100 samples per second
samples = 2**16   # for the FFT

Let's define a function to map the harmonic index (1 = fundamental, 2 = II harmonic, etc.) to the relative amplitude, according to different models.
For the _harp_ model, we use the coefficients derived in the first lesson:

$$C_n \propto \frac{1}{n^2} \sin(n \pi k), \quad k \in \ ]0, 0.5]$$

Where $k$ represents where the string is pinched, from the edge (0) to the middle (0.5) of the string. 

In [2]:
def htoamp(h, model, k=0.5):
    if model == 'flat':
        return 1
    if model == 'harp':
        return 1 / h ** 2 * sin(h * pi * k)
    if model == 'even':
        return 1 / h ** (2 if h % 2 > 0 else 1.1)   # even harmonics decade faster
    if model == 'odd':
        return 1 / h ** (2 if h % 2 == 0 else 1.1)    # odd harmonics decade faster
    if model == 'verotta':
        return 1 / h
    if model == 'sethares':
        return 0.8**h
    return 0

In [28]:
htoamp(1, 'harp', 3)

3.6739403974420594e-16

In [3]:
class Chord:
    def __init__(self, f, duration, harmonics, nonharm, onlyharm, nofundamental, h_model, k_harp):
        self.f = f
        self.duration = duration
        self.harmonics = harmonics
        self.onlyharm = onlyharm
        self.nofundamental = nofundamental
        self.nonharm = nonharm
        self.h_model = h_model
        self.k_harp = k_harp
    
    def note(self):
        '''Generate a numpy array for a given note index or frequency, where n = 0 means DO4.
        If onlyharm = True, generates only the harmonics.'''
        # generate array with duration*sample_rate steps, ranging between 0 and duration
        t = np.linspace(0, self.duration, int(self.duration * fs), False)

        if self.onlyharm:
            # generate only the requested harmonic
            w = np.sin(2 * np.pi * self.f * self.harmonics * t) * htoamp(self.harmonics, self.h_model, self.k_harp)
            return w

        # generate the fundamental wave. The damping is such that at t=duration, amplitude is about 0.3.
        w = (0 if self.nofundamental else 1) * np.sin(2 * np.pi * self.f * t) * htoamp(1, self.h_model, self.k_harp) * \
            (1 if 'harp' not in self.h_model else np.exp(- t * 1.2 / self.duration))
        # add the harmonics with given weights
        if self.harmonics > 1:
            for h in range(self.harmonics-1):   # h = 0..harmonics-2
                w += np.sin(2 * np.pi * self.f * (h+2) * (1+self.nonharm*random()) * t) * htoamp(h+2, self.h_model, self.k_harp) * \
                     (1 if 'harp' not in self.h_model else np.exp(- t * (h+2) * 1.2 / self.duration))
        
        return w


    def play(self, audio):
        '''Play a given signal to the audio card'''
        # ensure that highest value is in 16-bit range
        playable = audio * (2**15 - 1) / np.max(np.abs(audio))
        if self.onlyharm:
            # scale down so to hear the effect of the scaled single harmonic
            playable *= htoamp(self.harmonics, self.h_model, self.k_harp)
        # stop any ongoing play
        #sa.stop_all()
        # convert to 16-bit data and play
        return sa.play_buffer(playable.astype(np.int16), 1, 2, fs)

    def plottimefreq(self, s):
        # plot the time series
        plt.subplots(figsize=(15, 5)) 
        ax = plt.subplot(1, 2, 1)
        plt.plot(s)
        plt.xlim(0)
        ax.set_xlabel('t [ms]')
        maxx = int(self.duration*10 + 1)*100
        ax.set_xticks(np.arange(0, maxx*fs/1000, maxx*fs/10000, dtype=int))
        ax.set_xticklabels(np.arange(0, maxx, maxx/10, dtype=int))
        plt.grid(which='major')
        plt.title("Wave packet")

        # also compute and plot power spectrum
        ax = plt.subplot(1, 2, 2)
        s = np.pad(s, (0, samples-s.size), mode='constant')
        W = np.abs(np.fft.fft(s) ** 2)
        f = np.fft.fftfreq(s.size, 1/fs)
        plt.plot(f, W)
        plt.xlim(20, 15000)
        #formatter = LogFormatter(labelOnlyBase=False, minor_thresholds=(1, 0.1))
        #ax.get_xaxis().set_minor_formatter(formatter)
        ax.set_xlabel('f [Hz]')
        plt.ylim(1E-2)
        plt.xscale('log')
        plt.yscale('log')
        plt.grid(which='both')
        plt.title("Power spectrum (log/log)")
        plt.show()


In [4]:
@widgets.interact(
    frequency=widgets.IntSlider(min=100, max=500, value=220, continuous_update=False),
    duration=widgets.FloatSlider(min=0.01, max=1.48, value=0.7, step=0.01, continuous_update=False),
    harmonics=widgets.IntSlider(min=1, max=30, value=0, continuous_update=False),
    nonharm=widgets.FloatSlider(min=0, max=1, value=0, step=0.01, continuous_update=False),
    only_harm=widgets.Checkbox(description='only harmonic',value=False),
    no_fundamental=widgets.Checkbox(description='no fundamental',value=False),
    h_model=widgets.RadioButtons(options=['flat', 'harp']),
    k_harp=widgets.FloatSlider(min=0.01, max=0.5, value=0.5, step=0.01, continuous_update=False),
    window=widgets.RadioButtons(options=['rect', 'gaussian'])
    )
def interactiveplay(frequency, duration, harmonics, nonharm, only_harm, no_fundamental, h_model, k_harp, window):
    # generate signal for the given chord
    c = Chord(frequency, duration, harmonics, nonharm, only_harm, no_fundamental, h_model, k_harp)
    s = c.note()
    # use a window to smooth begin and end
    if window == 'hanning':
        s *= np.hanning(s.size)
    elif window == 'gaussian':
        s *= ss.gaussian(s.size, 5000*duration)
    c.plottimefreq(s)
    c.play(s)

interactive(children=(IntSlider(value=220, continuous_update=False, description='frequency', max=500, min=100)…